In [1]:
import os
from typing import List
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.callbacks import EarlyStopping
from torchvision import datasets, transforms
import torch
from torch.utils.data import DataLoader, sampler, random_split
import time
import copy
from tqdm import tqdm
from torch import nn, optim

2024-06-10 22:20:55.110798: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 22:20:55.187996: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 22:20:55.654782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 22:20:55.656991: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 22:21:00.940261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
original_dataset_path = os.path.join(".", "dataset")
class_names = sorted(os.listdir(os.path.join(original_dataset_path, "train")))
validation_dir = os.path.join(original_dataset_path, "valid")

to_train_dir = os.path.join(".", "pipes", "to-train")

train_gen = ImageDataGenerator(rescale=1./255)
train_data = train_gen.flow_from_directory(
  to_train_dir,
  target_size=(224,224),
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  classes=class_names,
)

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_directory(
  validation_dir,
  target_size=(224,224),
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  classes=class_names,
)

Found 11 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [3]:
callbacks = [EarlyStopping(monitor='accuracy' , patience=5 , restore_best_weights=True)]
model01 = keras.saving.load_model(os.path.join(".", "models", "modelo01.h5"))
history01 = model01.fit(
  train_data,
  epochs=max(1, int(len(train_data)/1000)),
  batch_size=32,
  callbacks=callbacks
)

1/1 [==============================] - 5s 5s/step - loss: 1.1921 - accuracy: 0.8182 - precision: 0.7500 - recall: 0.5455


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
class CustomImageFolder(datasets.ImageFolder):
  def __init__(self, root, transform=None, target_transform=None, classes=List[str]):
    super().__init__(root, transform=transform, target_transform=target_transform)
    self.class_to_idx = {c: i for i, c in enumerate(classes)}
    self.samples = [(path, self.class_to_idx[path.split('/')[-2]]) for path, target in self.samples if path.split('/')[-2] in self.class_to_idx]
    self.targets = [s[1] for s in self.samples]

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(), 
                                                transforms.GaussianBlur(3)]), p=0.1),
    transforms.Resize(256),
    transforms.CenterCrop(240),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.RandomErasing(p=0.14, value='random')
  ])
  train_data = CustomImageFolder(to_train_dir, transform=transform, classes=class_names)
  print(f"Found {len(train_data)} images for training with {len(train_data.classes)} classes")
  train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    model.train()

    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(train_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()

      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
    scheduler.step()

    epoch_loss = running_loss / len(train_data)
    epoch_acc = running_corrects.double() / len(train_data)

    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    if epoch_acc > best_acc:
      best_acc = epoch_acc
      best_model_wts = copy.deepcopy(model.state_dict())

  print()

  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best val Acc: {best_acc:4f}')

  model.load_state_dict(best_model_wts)
  return model

In [6]:
model02 = torch.jit.load(os.path.join(".", "models", "model02.zip"))

criterion = nn.CrossEntropyLoss(label_smoothing=0.11)
criterion = criterion.to(device)
optimizer = optim.AdamW(model02.classifier.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)
model02 = train_model(model02, criterion, optimizer, scheduler, max(1, int(len(train_data)/500)))

Found 11 images for training with 7 classes
Epoch 1/1
----------


100%|██████████| 1/1 [00:02<00:00,  2.10s/it]


Train Loss: 2.1970 Acc: 0.8182

Training complete in 0m 2s
Best val Acc: 0.818182


In [8]:
res = input("Você deseja salvar o modelo? [Y/N] ").lower()
if res == 'y':
  modelsPath = os.path.join(".", "models")
  os.rename(os.path.join(modelsPath, "modelo01.h5"), os.path.join(modelsPath, "old-modelo01.h5"))
  model01.save(os.path.join(modelsPath, "modelo01.h5"))

  scripted_model02 = torch.jit.script(model02)
  os.rename(os.path.join(modelsPath, "model02.zip"), os.path.join(modelsPath, "old-modelo02.zip"))
  torch.jit.save(scripted_model02, os.path.join(modelsPath, "modelo02.zip"))